In [93]:
import pandas as aju
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

from scipy.stats import zscore
from scipy.stats.mstats import winsorize

In [2]:
train = aju.read_csv('D:\\PYTHON CODINGS\\MACHINE LEARNING\\ML_00_PROGRAM FILES\\ADABOOST\\train.csv')
test = aju.read_csv('D:\\PYTHON CODINGS\\MACHINE LEARNING\\ML_00_PROGRAM FILES\\ADABOOST\\test.csv')

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train.shape

(891, 12)

In [5]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
test.shape

(418, 11)

In [7]:
df = aju.concat([train,test],sort = False)

In [8]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
df.shape

(1309, 12)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [11]:
df.Survived.value_counts()

Survived
0.0    549
1.0    342
Name: count, dtype: int64

In [12]:
df.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [13]:
df1 = df.drop('Cabin',axis = 1)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(4)
memory usage: 122.7+ KB


In [14]:
df1.Age.isnull().sum()

263

In [15]:
df1.Age.unique()

array([22.  , 38.  , 26.  , 35.  ,   nan, 54.  ,  2.  , 27.  , 14.  ,
        4.  , 58.  , 20.  , 39.  , 55.  , 31.  , 34.  , 15.  , 28.  ,
        8.  , 19.  , 40.  , 66.  , 42.  , 21.  , 18.  ,  3.  ,  7.  ,
       49.  , 29.  , 65.  , 28.5 ,  5.  , 11.  , 45.  , 17.  , 32.  ,
       16.  , 25.  ,  0.83, 30.  , 33.  , 23.  , 24.  , 46.  , 59.  ,
       71.  , 37.  , 47.  , 14.5 , 70.5 , 32.5 , 12.  ,  9.  , 36.5 ,
       51.  , 55.5 , 40.5 , 44.  ,  1.  , 61.  , 56.  , 50.  , 36.  ,
       45.5 , 20.5 , 62.  , 41.  , 52.  , 63.  , 23.5 ,  0.92, 43.  ,
       60.  , 10.  , 64.  , 13.  , 48.  ,  0.75, 53.  , 57.  , 80.  ,
       70.  , 24.5 ,  6.  ,  0.67, 30.5 ,  0.42, 34.5 , 74.  , 22.5 ,
       18.5 , 67.  , 76.  , 26.5 , 60.5 , 11.5 ,  0.33,  0.17, 38.5 ])

In [16]:
si = SimpleImputer(strategy = 'mean')
df1.Age = si.fit_transform(df1[['Age']])

In [17]:
df1.Age.isnull().sum()

0

In [18]:
df1.Survived = si.fit_transform(df1[['Survived']])

In [19]:
df1.Survived.isnull().sum()

0

In [20]:
df1.Embarked.isnull().sum()

2

In [21]:
si1 = SimpleImputer(strategy = 'most_frequent')
df1['Embarked'] = si1.fit_transform(df1[['Embarked']]).ravel()

In [22]:
df1.Embarked.isnull().sum()

0

In [23]:
df1.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           1
Embarked       0
dtype: int64

In [24]:
df1['Fare'] = si1.fit_transform(df1[['Fare']]).ravel()

In [25]:
df1.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [26]:
df2 = df1.drop(['Name','Ticket'],axis = 1)

In [27]:
threshold = 3
features  = ['Age','Fare']
zscores = np.abs(zscore(df2[features]))
zscores

,Age,Fare
0,0.611972,0.503176
1,0.630431,0.734809
2,0.301371,0.490126
3,0.397481,0.383263
4,0.397481,0.487709
...,...,...
413,0.000000,0.487709
414,0.708081,1.462069
415,0.669256,0.503176
416,0.000000,0.487709


In [28]:
outliers = np.where(zscores>threshold)
outliers

(array([  27,   88,   96,  116,  118,  258,  299,  311,  341,  377,  380,
         438,  493,  527,  557,  630,  672,  679,  689,  700,  716,  730,
         737,  742,  745,  779,  851,  915,  944,  950,  955,  960,  965,
         966,  972,  987, 1005, 1033, 1047, 1075, 1093, 1109, 1215, 1234,
        1266, 1298], dtype=int64),
 array([1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1], dtype=int64))

In [29]:
df2['Age']= winsorize(df2['Age'],limits=[0.15,0.15] )
df2['Fare']= winsorize(df2['Fare'],limits=[0.15,0.15] )

In [30]:
features  = ['Age','Fare']
zscores = np.abs(zscore(df2[features]))
outliers = np.where(zscores>threshold)
outliers

(array([], dtype=int64), array([], dtype=int64))

In [61]:
df2.shape

(1309, 9)

In [65]:
oe = OrdinalEncoder()
df2['Sex'] = oe.fit_transform(df2[['Sex']])
df2['Embarked'] = oe.fit_transform(df2[['Embarked']])

In [71]:
df2['Survived'] = oe.fit_transform(df2[['Survived']])

In [75]:
x = df2.drop('Survived',axis = 1)
y = df2.Survived

In [79]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state = 99)

In [97]:
model = AdaBoostClassifier(n_estimators = 100,random_state = 0 )

In [99]:
model.fit(x_train,y_train)

D:\python_anaconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(n_estimators=100, random_state=0)

In [101]:
y_pred = model.predict(x_test)

In [103]:
model.score(x_test,y_test)

0.7608142493638677

In [105]:
accuracy_score(y_pred,y_test)

0.7608142493638677